In [1]:
import numpy as np
from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile
from pynwb.ecephys import ElectricalSeries
from pynwb import NWBHDF5IO

from process_nwb import signal_processing
from process_nwb.signal_processing import (common_referencing, wavelet_transform,
                                           resample, linenoise_notch)
from imp import reload

# Create raw file

In [2]:
rm ecephys_example.nwb

In [3]:
nwbfile = NWBFile('my first synthetic recording', 'EXAMPLE_ID', datetime.now(tzlocal()),
                  experimenter='JL',
                  lab='home',
                  institution='inst',
                  experiment_description='desc',
                  session_id='sess')

device = nwbfile.create_device(name='trodes_rig123')
electrode_name = 'ecog'
description = "an example tetrode"
location = "somewhere in the hippocampus"

electrode_group = nwbfile.create_electrode_group(electrode_name,
                                                 description=description,
                                                 location=location,
                                                 device=device)

for idx in range(8):
    nwbfile.add_electrode(idx,
                          x=1.0, y=2.0, z=3.0,
                          imp=float(-idx),
                          location='CA1', filtering='none',
                          group=electrode_group)
electrode_table_region = nwbfile.create_electrode_table_region(np.arange(8).tolist(), 'all electrodes')

rate = 3000.0
np.random.seed(1234)
data_len = 10000
ephys_data = np.random.rand(data_len, 8)

ephys_ts = ElectricalSeries('test_ephys_data',
                            ephys_data,
                            electrode_table_region,
                            starting_time=0.,
                            rate=rate,
                            description="Random numbers generated with numpy.random.rand")
nwbfile.add_acquisition(ephys_ts)

with NWBHDF5IO('ecephys_example.nwb', 'w') as io:
    io.write(nwbfile)

# Add preprocessed data

In [11]:
reload(resample)
reload(linenoise_notch)
reload(common_referencing)
reload(wavelet_transform)

with NWBHDF5IO('ecephys_example.nwb', 'r+') as io:
    nwbfile = io.read()
    X_raw = nwbfile.acquisition['test_ephys_data'].data[:]
    X_ds, es_ds = resample.store_resample(nwbfile, 'test_ephys_data', 400.)
    X_CAR_ln, es_car_ln = signal_processing.store_linenoise_notch_store_CAR(nwbfile, es_ds.name)
    X_wvlt, es_wvlt = wavelet_transform.store_wavelet_transform(nwbfile, es_car_ln.name)
    print(nwbfile.acquisition)
    #io.write(nwbfile)

{'test_ephys_data': test_ephys_data pynwb.ecephys.ElectricalSeries at 0x121004066968
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (10000, 8), type "<f8">
  description: Random numbers generated with numpy.random.rand
  electrodes: electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  rate: 3000.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: volts
, 'downsampled_test_ephys_data': downsampled_test_ephys_data pynwb.ecephys.ElectricalSeries at 0x120912939160
Fields:
  comments: no comments
  conversion: 1.0
  data: [[-0.28229176  6.04266374  2.9245181  ...  0.64886129  1.299189
   7.19773621]
 [ 4.2258843   2.66751669  3.56334478 ...  4.05025707  3.37043417
   4.13522136]
 [ 3.95106591  3.92290891  3.01689171 ...  3.45365994  4.04112758
   4.00357076]
 ...
 [ 3.49070049  3.86804485  2.77247089 ...  3.89532146  3.22471445
   3.04037627]
 [ 4.04305563  3.86854665  5.18127742 ...  3.91765298  3.13216091
   2.77122